# Submission code with comments (QunaVillage)

## Basic Idea

### Saving number of qubits via constraints
Node 2 is constrained to take either B or D, due to the adjacent nodes.  
So we assign only one qubit for node 2 as B=|0>, D=|1>.  

On the other hand, if we put D on node 0, node 2, 3, and 1 are   automatically fixed, but node 6 becomes inconsistent eventually.  
Thus we can only put B or C on node 0 (assume B=|0>, C=|1>).  

In the same way, node 1 is constrained to either A(=|0>) or D(=|1>).

Consequently, non-trivial edges are as follows.  
We number them consecutively.

```
edge 0 = (node A, node 3)  # and so forth
edge 1 = (0, 2)  
edge 2 = (0, 3)  
edge 3 = (1, 3)  
edge 4 = (1, 4)  
edge 5 = (B, 4)  
edge 6 = (2, 3)  
edge 7 = (3, 4)  
edge 8 = (2, 5)  
edge 9 = (2, 6)  
edge 10 = (3, 5)  
edge 11 = (3, 6)  
edge 12 = (4, 6)  
edge 13 = (5, 6)  
edge 14 = (5, D)  
edge 15 = (6, D)
```

### Dividing edges to seven parts
Furthermore, we divide these edges to seven groups which we call "parts".   
Edges in the same part will be judged if the qubits meet the condition at the same time.
```
part0 = edge 1  # and so forth
part1 = 0
part2 = (2,3)
part3 = (4,5)
part4 = (6,8,9,10)
part5 = (7,11,12)
part6 = (13,14,15)
```

### Table of qubit usages
In total, we use 27 qubits.

|qubit|node|
|---|---|
|0|0 (one qubit to represent B/C)| 
|1|1 (one qubit to represent A/D)|
|2|2 (one qubit to represent B/D)|
|3|3 (the right digit of node 3 e.g., \|1> if \|node3>=\|B>=\|01> and so forth) |
|4|3 (the left digit of node 3 e.g., \|0> if \|node3>=\|B>=\|01> and so forth) |
|5|4|
|6|4|
|7|5|
|8|5|
|9|6|
|10|6|
|11| target for part0 and diffusion MCT ancilla 0|
|12| target for part1 and diffusion MCT ancilla 1|
|13| target for part2 and diffusion MCT ancilla 2|
|14| target for part3 and diffusion MCT ancilla 3|
|15| target for part4 and diffusion MCT ancilla 4|
|16| target for part4 and diffusion MCT ancilla 5|
|17| target for part5 and diffusion MCT ancilla 6|
|18| target for part6 and diffusion MCT ancilla 7|
|19|part4, ancillary|
|20|part5, ancillary|
|21|part6, ancillary|
|22|phase MCT ancilla0|
|23|phase MCT ancilla1|
|24|phase MCT ancilla2|
|25|phase MCT ancilla3|
|26|phase MCT ancilla4|

### Other points
* Using RCCX gates (Relative-phase Toffoli gates) instead of CCX to save CX gates
* Thinking carefully the order to apply gates to reduce redundant gate operations, especially parts gates for the oracle.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import time 

from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit import IBMQ, Aer, execute
from qiskit.tools.visualization import plot_histogram
import json
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Unroller

In [2]:
def part0(circuit, q0, q1, t0):
    """
    for nodes 0, 2
    :param circuit:
    :param q0: node 0
    :param q1: node 2
    :param t0: target
    :return:
    """
    circuit.x(q0)
    circuit.x(q1)
    circuit.rccx(q0, q1, t0)
    circuit.x(t0)

def part0_inv(circuit, q0, q1, t0):
    circuit.x(t0)
    circuit.rccx(q0, q1, t0)
    circuit.x(q1)
    circuit.x(q0)
    

def part1(circuit, q0, q1, t0):
    """
    for nodes A, 3
    :param circuit: 
    :param q0: node 3
    :param q1: node 3
    :param t0: target
    :return: 
    """
    circuit.x(q0)
    circuit.x(q1)
    circuit.rccx(q0, q1, t0)
    circuit.x(t0)
    circuit.x(q1)
    circuit.x(q0)

def part1_inv(circuit, q0, q1, t0):
    circuit.x(q0)
    circuit.x(q1)
    circuit.x(t0)
    circuit.rccx(q0, q1, t0)
    circuit.x(q1)
    circuit.x(q0)
    

def part2(circuit, q0, q1, q2, q3, t0):
    """
    for nodes 0, 1, 3
    :param circuit:
    :param q0: node 0 (|0> = B, |1> = C)
    :param q1: node 1 (|0> = A, |1> = D)
    :param q2: node 3
    :param q3: node 3
    :param t0:
    :return:
    """
    # q0 XOR q1 -> q0
    circuit.cx(q1, q0)

    # cswap
    circuit.cx(q2, q3)
    circuit.rccx(q0, q3, q2)
    circuit.cx(q2, q3)

    circuit.cx(q1, t0)
    circuit.cx(q3, t0) 

    circuit.cx(q2, q3)
    circuit.rccx(q0, q3, q2)
    circuit.cx(q2, q3)

    circuit.cx(q1, q0)

def part2_inv(circuit, q0, q1, q2, q3, t0):
    circuit.cx(q1, q0)

    circuit.cx(q2, q3)
    circuit.rccx(q0, q3, q2)
    circuit.cx(q2, q3)

    circuit.cx(q3, t0)
    circuit.cx(q1, t0)

    circuit.cx(q2, q3)
    circuit.rccx(q0, q3, q2)
    circuit.cx(q2, q3)

    circuit.cx(q1, q0)

    
def part3(circuit, q0, q1, q2, t0):
    """
    for nodes 1, 4, B
    :param circuit:
    :param q0: node 1
    :param q1: node 4
    :param q2: node 4
    :param t0: target
    :return:
    """
    circuit.x(q1)
    circuit.rccx(q0, q1, t0)
    circuit.x(q0)
    circuit.rccx(q0, q2, t0)

def part3_inv(circuit, q0, q1, q2, t0):
    circuit.rccx(q0, q2, t0)
    circuit.x(q0)
    circuit.rccx(q0, q1, t0)
    circuit.x(q1)
    
    
def part4(circuit, q0, q1, q2, q3, q4, q5, q6, t0, t1, a0):
    """
    for the triangle (cons. of nodes 2, 3, 5) and the edge (cons. of nodes 2, 6)

    :param circuit: 
    :param q0: node 2
    :param q1: node 3
    :param q2: node 3
    :param q3: node 5
    :param q4: node 5
    :param q5: node 6
    :param q6: node 6
    :param t0: target for triangle (cons. of nodes 2, 3, 5)
    :param t1: target for the edge (cons. of nodes 2, 6)
    :param a0: ancillary qubit
    :return: 
    """

    ### triangle
    circuit.x(q1)
    circuit.x(q3)
    circuit.cx(q0, q2)
    circuit.cx(q0, q4)

    circuit.rccx(q2, q3, t0)
    circuit.rccx(q1, q4, t0)
    circuit.rccx(q1, q3, t0)
    circuit.rccx(q2, q4, t0)
    circuit.rccx(q1, q3, a0)
    circuit.rccx(q2, q4, a0)
    circuit.cx(a0, t0)

    ### edge
    circuit.cx(q0, q6)
    circuit.x(q6)
    circuit.rccx(q5, q6, t1)
    circuit.x(t1)

def part4_inv(circuit, q0, q1, q2, q3, q4, q5, q6, t0, t1, a0):

    circuit.x(t1)
    circuit.rccx(q5, q6, t1)
    circuit.x(q6)
    circuit.cx(q0, q6)

    circuit.cx(a0, t0)
    circuit.rccx(q2, q4, a0)
    circuit.rccx(q1, q3, a0)
    circuit.rccx(q2, q4, t0)
    circuit.rccx(q1, q3, t0)
    circuit.rccx(q1, q4, t0)
    circuit.rccx(q2, q3, t0)

    circuit.cx(q0, q4)
    circuit.cx(q0, q2)
    circuit.x(q3)
    circuit.x(q1)


def part5(circuit, q0, q1, q2, q3, q4, q5, t0, a0):
    """
    for nodes 3, 4, 6
    :param circuit:
    :param q0: node 3
    :param q1: node 3
    :param q2: node 4
    :param q3: node 4
    :param q4: node 6
    :param q5: node 6
    """
    circuit.cx(q0, q2)
    circuit.cx(q0, q4)
    circuit.cx(q1, q3)
    circuit.cx(q1, q5)

    circuit.rccx(q3, q4, t0)
    circuit.rccx(q2, q5, t0)
    circuit.rccx(q2, q4, t0)
    circuit.rccx(q3, q5, t0)
    circuit.rccx(q2, q4, a0)
    circuit.rccx(q3, q5, a0)
    circuit.cx(a0, t0)

    #circuit.barrier()
    circuit.cx(q1, q5)
    circuit.cx(q1, q3)
    circuit.cx(q0, q4)
    circuit.cx(q0, q2)

def part5_inv(circuit, q0, q1, q2, q3, q4, q5, t0, a0):
    circuit.cx(q0, q2)
    circuit.cx(q0, q4)
    circuit.cx(q1, q3)
    circuit.cx(q1, q5)
    #circuit.barrier()

    circuit.cx(a0, t0)
    circuit.rccx(q3, q5, a0)
    circuit.rccx(q2, q4, a0)
    circuit.rccx(q3, q5, t0)
    circuit.rccx(q2, q4, t0)
    circuit.rccx(q2, q5, t0)
    circuit.rccx(q3, q4, t0)
    
    circuit.cx(q1, q5)
    circuit.cx(q1, q3)
    circuit.cx(q0, q4)
    circuit.cx(q0, q2)

    
def part6(circuit, q0, q1, q2, q3, t0, a0):
    """
    for nodes 5, 6, D
    :param circuit:
    :param q0: node 5
    :param q1: node 5
    :param q2: node 6
    :param q3: node 6
    """
    circuit.x(q0)
    circuit.x(q1)
    circuit.x(q2)
    circuit.x(q3)
    circuit.rccx(q1, q2, t0)
    circuit.rccx(q0, q3, t0)
    circuit.rccx(q0, q2, t0)
    circuit.rccx(q1, q3, t0)
    circuit.rccx(q0, q2, a0)
    circuit.rccx(q1, q3, a0)
    circuit.cx(a0, t0)

    circuit.x(q3)
    circuit.x(q2)
    circuit.x(q1)
    circuit.x(q0)

def part6_inv(circuit, q0, q1, q2, q3, t0, a0):
    circuit.x(q0)
    circuit.x(q1)
    circuit.x(q2)
    circuit.x(q3)
    
    circuit.cx(a0, t0)
    circuit.rccx(q1, q3, a0)
    circuit.rccx(q0, q2, a0)
    circuit.rccx(q1, q3, t0)
    circuit.rccx(q0, q2, t0)
    circuit.rccx(q0, q3, t0)
    circuit.rccx(q1, q2, t0)
    circuit.x(q3)
    circuit.x(q2)
    circuit.x(q1)
    circuit.x(q0)

In [3]:
def phase(circuit, q0, q1, q2, q3, q4, q5, q6, q7, a0, a1, a2, a3, a4):
    """
    q0, q1, ..., q7: target qubits for part0 ~ part6 (note that part4 has two targets)
    We multiply phase factor (-1) when q0=q1=...=q7=1, i.e., all edges satisfy the condition.
    """
    circuit.h(q7)
    circuit.mct([q0, q1, q2, q3, q4, q5, q6], q7, [a0, a1, a2, a3, a4], mode="basic")
    circuit.h(q7)

In [4]:
def oracle(circuit, q):
    
    # test each part -> encode result to target qubits
    # the order of applying each part is decided so that the number of qubits 
    # which have to be applied inverse-operation to be minimized
    part1(circuit, q[3], q[4],  q[12])  
    part2(circuit, q[0], q[1], q[3], q[4], q[13]) 
    part5(circuit, q[3], q[4], q[5], q[6], q[9], q[10], q[17], q[20])
    part6(circuit, q[7], q[8], q[9], q[10], q[18], q[21])
    part3(circuit, q[1], q[5], q[6], q[14])
    part4(circuit, q[2], q[3], q[4], q[7], q[8], q[9], q[10], q[15], q[16], q[19])  
    part0(circuit, q[0], q[2], q[11])

    # phase flip
    phase(circuit, 
          q[11], q[12], q[13], q[14], q[15], q[16], q[17], q[18], 
          q[22], q[23], q[24], q[25], q[26]) 
    
    # apply inverse-operation for each part
    part0_inv(circuit, q[0], q[2], q[11])
    part4_inv(circuit, q[2], q[3], q[4], q[7], q[8], q[9], q[10], q[15], q[16], q[19])
    part3_inv(circuit, q[1], q[5], q[6], q[14])
    part6_inv(circuit, q[7], q[8], q[9], q[10], q[18], q[21])
    part5_inv(circuit, q[3], q[4], q[5], q[6], q[9], q[10], q[17], q[20])
    part2_inv(circuit, q[0], q[1], q[3], q[4], q[13]) 
    part1_inv(circuit, q[3], q[4],  q[12])  

In [5]:
def diffusion(circuit, qr, anc):
    """
    diffusion (inversion about the mean) circuit.
    note that this implementation gives H^{\otimes n} (Id - |0..0><0...0|) H^{\otimes n} 
    :param circuit:
    :param qr: QuantumRegister on nodes
    :param anc: ancillary qubits
    :return:
    """
    circuit.h(qr)
    circuit.x(qr)

    # apply multi-control CZ
    circuit.h(qr[-1])
    circuit.mct(qr[:-1], qr[-1], anc, mode='basic')
    circuit.h(qr[-1])

    circuit.x(qr)
    circuit.h(qr)

In [6]:
def map2c(circuit, qs, cs, a0, a1):
    """
    measurement of qubits so that the result is stored on c-bits,
    following the required format. 
    """
    circuit.x(a0)
    circuit.x(a1)
    circuit.cx(qs[0], a1)
    circuit.measure([qs[0], a1, qs[1], qs[1], qs[2], a0, qs[3], qs[4], qs[5], qs[6], qs[7], qs[8], qs[9], qs[10]], 
            [cs[0], cs[1], cs[2], cs[3], cs[4], cs[5], cs[7], cs[6], cs[9], cs[8], cs[11], cs[10], cs[13], cs[12]])

In [7]:
def grover_single_iter(circuit, q):
    oracle(circuit, q)
    diffusion(circuit, q[:11], q[11:(11+8)])

In [8]:
# circuit construction 
q = QuantumRegister(27) 
c = ClassicalRegister(14)
circuit = QuantumCircuit(q, c)

## prepare superposition
circuit.h(q[:11])

## Grover iteration
for _ in range(5):
    grover_single_iter(circuit, q)

# measurement and formatting
map2c(circuit, q[:11], c[:14], q[11], q[12])

In [9]:
# Unroll the circuit
pass_ = Unroller(['u3', 'cx'])
pm = PassManager(pass_)
new_circuit = pm.run(circuit) 

# obtain gates
gates=new_circuit.count_ops()
print(gates)
cost = gates["u3"] + 10 * gates["cx"]
print("cost:", cost)

OrderedDict([('u3', 2843), ('cx', 1421), ('measure', 14)])
cost: 17053


In [10]:
### Execution
time0 = time.time()
shots = 8000
print("shots:", shots)

## local simulator
# backend = Aer.get_backend('qasm_simulator')
## cloud simulator
provider = IBMQ.load_account()
backend = provider.get_backend('ibmq_qasm_simulator')

job = execute(circuit, backend=backend, shots=shots, seed_simulator=12345, backend_options={"fusion_enable":True})
result = job.result()
time1 = time.time()
print(f"Calculations took {time1-time0} sec.")

shots: 8000
Calculations took 99.07090997695923 sec.


In [11]:
###  ----------- Data Formatting -------------
# Input your quantum circuit
circuit = circuit
# Input your result of the execute(groverCircuit, backend=backend, shots=shots).result()
results = result
count = results.get_counts()
# Input your name or your team name
name='QunaVillage'
# Please indicate the number of times you have submitted your answer so far.
# For example, If it's your 1st time to submit your answer, write times='1'. If it's your 5th time to submit your answer, write times='5'.
# Do not forget to put single quotations to make the variable 'Str'' type
times= '3'

# Unroll the circuit
pass_ = Unroller(['u3', 'cx'])
pm = PassManager(pass_)
new_circuit = pm.run(circuit) 

# obtain gates
gates=new_circuit.count_ops()

#sort count
count_sorted = sorted(count.items(), key=lambda x:x[1], reverse=True)

# collect answers with Top 9 probability
ans_list = count_sorted[0:9]

# reverse ans_list
ans_reversed = []
for i in ans_list:
    ans_temp=[i[0][::-1],i[1]]
    ans_reversed.append(ans_temp)

# convert each 2 bits into corresponding color. Add node0(0),node3(1),node8(2) and node11(3)
ans_shaped = []
for j in ans_reversed:
    ans_temp=j[0]
    nodeA = 0
    node0 = int(ans_temp[0] + ans_temp[1], 2)
    node1 = int(ans_temp[2] + ans_temp[3], 2)
    nodeB = 1
    node2 = int(ans_temp[4] + ans_temp[5], 2)
    node3 = int(ans_temp[6] + ans_temp[7], 2)
    node4 = int(ans_temp[8] + ans_temp[9], 2)
    nodeC = 2
    node5 = int(ans_temp[10] + ans_temp[11], 2)
    node6 = int(ans_temp[12] + ans_temp[13], 2)
    nodeD = 3
    nodes_color = str(nodeA) + str(node0) + str(node1) + str(nodeB) + str(node2) + str(node3) + str(node4) + str(nodeC) + str(node5) + str(node6) + str(nodeD) 
    ans_shaped.append([nodes_color,j[1]])

# write the result into '[your name]_final_output.txt'
filename = name +'_'+times+'_final_output.txt'
dct={'ans':ans_shaped,'costs':gates}
with open(filename, 'w') as f:
    json.dump(dct, f)

###  ----------- verification -------------
from pathlib import Path
your_path = filename
p= Path(your_path)

# Verify your information
f_name=p.name
your_info=f_name.split('_')
print('Your name: ', your_info[0])
print('The number of times you have submitted your answer: ', your_info[1])

with open(p, 'r') as f:
    print(f)
    your_ans=json.load(f)

print('Does your submission file have 9 answers?')
if (len(your_ans['ans'])!=9):
    print('- No, make sure you have 9 answers with top 9 probabilities')
else:
    print('- Yes')
    print('- Your plan: ', your_ans['ans'])

print('What is your cost?')
your_cost=your_ans['costs']['u3'] + 10*your_ans['costs']['cx']
print('- ', your_cost)

Your name:  QunaVillage
The number of times you have submitted your answer:  3
<_io.TextIOWrapper name='QunaVillage_3_final_output.txt' mode='r' encoding='UTF-8'>
Does your submission file have 9 answers?
- Yes
- Your plan:  [['02013132203', 425], ['01013232013', 407], ['02313102023', 402], ['01313202013', 402], ['02013132023', 396], ['01013232103', 391], ['02011322203', 388], ['02313122203', 388], ['02013122203', 386]]
What is your cost?
-  17053
